In [1]:
"""This is my 'data acquisition' notebook"""

"This is my 'data acquisition' notebook"

In [183]:
"""Imports"""
import requests
from bs4 import BeautifulSoup
import re
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [153]:
def get_property_data(property_url):
    """This function will receive the url of the property and parse its contents. It will then save the property's data in a dictionary."""
    property_headers = []
    property_data = []
    property = requests.get(property_url).text
    property_soup = BeautifulSoup(property, "html.parser")
    for code in property_soup.find("div", attrs={"class": "classified__header--immoweb-code"}):
        code_string = str(code)
        code_strip = code_string.strip()
        code_split = code_strip.split(":",)
        property_headers.append(code_split[0])
        property_data.append(code_split[1])
    for price in property_soup.find("p", attrs={"class": "classified__price"}):
        raw_price = str(price)
        sub_price = re.sub(r"<.*?>", '', raw_price).strip()
        split_price = sub_price.split()
        for list_price in split_price:
            if len(list_price) > 0:
                property_headers.append("Price")
                property_data.append(list_price)
    for address in property_soup.find("script", attrs={"type": "text/javascript"}):
        string_address = str(address)
        sub_address = re.search("\"country\"(.*?),\"distance\"", string_address).group()
        split_address = re.split(":|,", sub_address)
        split_address.remove('"distance"')
        list_address = [item.replace('"', '') for item in split_address]
        address_headers = list_address[0:len(list_address):2]
        for ah in address_headers:
            property_headers.append(ah)
        address_data = list_address[1:len(list_address):2]
        for ad in address_data:
            property_data.append(ad)
    for tr in property_soup.find_all("tr"):
        for th in tr.find_all("th"):
            header = str(th.string)
            header_strip = header.strip()
            if header_strip != 'None':
                property_headers.append(header_strip)
        for td in tr.find_all("td"):
            raw_datum = str(td)
            sub_datum_1 = re.sub(r"<.*?>", '', raw_datum).strip()
            sub_datum_2 = re.sub(r"\n", '', sub_datum_1).strip()
            sub_datum_3 = re.sub(r"\s{2,}", '', sub_datum_2).strip()
            if sub_datum_3 != '':
                property_data.append(sub_datum_3)
    property_dictionary = dict(zip(property_headers, property_data))
    return (property_dictionary)

In [163]:
##Check
get_property_data("https://www.immoweb.be/en/classified/new-real-estate-project-houses/for-sale/saint-sauveur/7912/10153840")

{'Immoweb code ': ' 10153840',
 'Price': '417000€',
 'country': 'Belgium',
 'region': 'null',
 'province': 'Hainaut',
 'district': 'Soignies',
 'locality': 'Lessines',
 'postalCode': '7860',
 'street': "Porte d'Ogy 78",
 'number': 'null',
 'box': 'null',
 'propertyName': 'null',
 'floor': 'null',
 'latitude': '50.71',
 'longitude': '3.83',
 'Primary energy consumption': 'Not specified',
 'Energy class': 'Not specified',
 'Reference number of the EPC report': 'Not specified',
 'CO₂ emission': 'Not specified',
 'Yearly theoretical total energy consumption': 'Not specified',
 'Tenement building': 'No',
 'Address': "Porte d'Ogy 787860- Lessines",
 'Website': 'http://www.immobeguin.be'}

In [205]:
def get_property_links(page_url):
    """This function will get and save all the url links of properties in a page given the url of the page"""
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(page_url)
    driver.find_element("id", "uc-btn-accept-banner").click()
    page_soup = BeautifulSoup(driver.page_source, "html.parser")
    ##driver.close()

In [204]:
get_property_links("https://www.immoweb.be/en/search/apartment/for-sale?countries=BE")

C:\Users\ARCH GLOBAL\AppData\Local\Temp\ipykernel_14764\111090840.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
